In [8]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
import random as rn
from tensorflow import set_random_seed
import matplotlib.pyplot as plt
%matplotlib inline
import os
cwd = os.getcwd()

In [9]:
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

In [10]:
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

In [11]:
# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

In [12]:
from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

In [13]:

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
class NeuralNet:

    def __init__(self, nClasses, randomSeed, epochs):
        self.images = None
        self.labels = None
        self.reshapedImages = None
        self.categoricalLabels = None
        self.nClasses = nClasses
        self.testingDataIndexes = []
        self.trainingValidationDataIndexes = []
        self.dataSize = 0
        self.trainingSetSize = 0
        self.validationSetSize = 0
        self.testingSetSize = 0
        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None
        self.randomSeed = randomSeed
        self.loss = np.zeros(epochs)
        self.acc = np.zeros(epochs)
        self.val_loss = np.zeros(epochs)
        self.val_acc = np.zeros(epochs)
        self.epochs = epochs
        self.model=None


    def loadData(self):
        self.images = np.load('images.npy')
        self.labels = np.load('labels.npy')

    def normalize(self):
        #self.reshapedImages = tf.keras.utils.normalize(self.reshapedImages, axis=1)
        self.reshapedImages /= 225
    def processData(self):
        count = 0
        self.dataSize = len(self.images)
        self.reshapedImages = np.zeros((self.dataSize, len(self.images[0].reshape(-1))))
        self.categoricalLabels = np.zeros((self.dataSize, self.nClasses))
        while count < self.dataSize:
            self.reshapedImages[count] = self.images[count].reshape(-1)
            self.categoricalLabels[count] = to_categorical(self.labels[count], num_classes = self.nClasses, dtype = 'float32')
            count += 1

    def calculateDataSetSizes(self, training, validation):
        self.trainingSetSize = int(self.dataSize * training)
        self.validationSetSize = int(self.dataSize * validation)
        self.testingSetSize = self.dataSize - (self.trainingSetSize + self.validationSetSize)

    def getDataSets(self, testing):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.reshapedImages, self.labels, test_size=testing, random_state=1, stratify=self.categoricalLabels)



    def runModel(self, x_train, y_train, x_val, y_val):
        # Model Template
        model = Sequential() # declare model
        model.add(Dense(100, input_shape=(28*28, ), activation = 'relu')) # first layer
        # Hidden Layer 1
        model.add(Dense(100, activation = 'relu'))
        # Hidden Layer 2
        #
        # Fill in Model Here
        # Hidden Layer 3
        #model.add(Dense(15, activation = 'relu'))

        # Hidden Layer 4
        #model.add(Dense(15, activation = 'relu'))

        # Hidden Layer 5
        #         model.add(Dense(75, activation = 'relu'))
        #
        model.add(Dense(10, activation = 'softmax')) # last layer


        # Compile Model
        model.compile(optimizer='sgd',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Train Model
        history = model.fit(x_train, y_train,
                            validation_data = (x_val, y_val),
                            epochs=self.epochs,
                            batch_size=80)
        self.model=model

        #Save model
        import os
        save_dir = cwd
        model_name = 'trained_model.proj3'
        model_path = os.path.join(save_dir, model_name)
        self.model.save(model_path)
        
        # Report Results
        print(history.history)
        return history

    def predictModel(self):
        y_pred=model.predict(X_test)

    def performCrossVal(self):
        kfold = StratifiedKFold(n_splits = 5, random_state = self.randomSeed)
        histories = []
        cat_y_train = to_categorical(self.y_train, num_classes = self.nClasses, dtype = 'float32')
        for train, validate in kfold.split(self.x_train, self.y_train):
            history = self.runModel(self.x_train[train], cat_y_train[train], self.x_train[validate], cat_y_train[validate])
            histories.append(history)

        for hist in histories:
            n = 0
            while n < self.epochs:
                self.loss[n] += hist.history['loss'][n]
                self.acc[n] += hist.history['acc'][n]
                self.val_loss[n] += hist.history['val_loss'][n]
                self.val_acc[n] += hist.history['val_acc'][n]
                n += 1

        self.loss = np.true_divide(self.loss, len(histories))
        self.acc = np.true_divide(self.acc, len(histories))
        self.val_loss = np.true_divide(self.val_loss, len(histories))
        self.val_acc = np.true_divide(self.val_acc, len(histories))

        print("self.loss = " + str(self.loss))
        print("self.acc = " + str(self.acc))
        print("self.val_loss = " + str(self.val_loss))
        print("self.val_acc = " + str(self.val_acc))
        
    def predict(self):
        # load the model and create predictions on the test set
        mnist_model = load_model("trained_model.proj3")
        predicted_classes = mnist_model.predict_classes(self.x_test)

        # see which we predicted correctly and which not
        correct_indices = np.nonzero(predicted_classes == self.y_test)[0]
        incorrect_indices = np.nonzero(predicted_classes != self.y_test)[0]
        print()
        print(len(correct_indices)," classified correctly")
        print(len(incorrect_indices)," classified incorrectly")

        # adapt figure size to accomodate 18 subplots
        plt.rcParams['figure.figsize'] = (7,14)

        figure_evaluation = plt.figure()

        # plot 9 correct predictions
        for i, correct in enumerate(correct_indices[:9]):
            plt.subplot(6,3,i+1)
            plt.imshow(self.x_test[correct].reshape(28,28), cmap='gray', interpolation='none')
            plt.title(
              "label:{}, Truth:{}".format(predicted_classes[correct],
                                                self.y_test[correct]))
            plt.xticks([])
            plt.yticks([])

        # plot 9 incorrect predictions
        for i, incorrect in enumerate(incorrect_indices[:9]):
            plt.subplot(6,3,i+10)
            plt.imshow(self.x_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
            plt.title(
              "label:{}, Truth:{}".format(predicted_classes[incorrect], 
                                               self.y_test[incorrect]))
            plt.xticks([])
            plt.yticks([])
        figure_evaluation
        
    def plot_metrics(self):
        # plotting the metrics
        fig = plt.figure()
        plt.subplot(2,1,1)
        plt.plot(self.acc)
        plt.plot(self.val_acc)
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='lower right')
        plt.tight_layout()
        fig.show


if __name__ == '__main__':
    train = 0.6
    validate = 0.15
    test = 0.25
    n_classes = 10
    randomSeed = 1
    epochs = 20
    np.random.seed(randomSeed)
    #main(train, validate, test, n_classes)

    net = NeuralNet(n_classes, randomSeed, epochs)
    net.loadData()
    net.processData()
    net.normalize()
    net.calculateDataSetSizes(train, validate)
    net.getDataSets(test)
    net.performCrossVal()
    net.predictModel
    net.predict()
    net.plot_metrics()

#results = model.predict(x_test)

Train on 3896 samples, validate on 979 samples
Epoch 1/20
3896/3896 [==============================] - 3s 810us/step - loss: 2.2257 - acc: 0.2346 - val_loss: 2.0920 - val_acc: 0.4147
Epoch 2/20
3896/3896 [==============================] - 2s 505us/step - loss: 1.9429 - acc: 0.5118 - val_loss: 1.8046 - val_acc: 0.5832
Epoch 3/20
3896/3896 [==============================] - 2s 559us/step - loss: 1.6268 - acc: 0.6512 - val_loss: 1.4847 - val_acc: 0.6864
Epoch 4/20
3896/3896 [==============================] - 2s 529us/step - loss: 1.3086 - acc: 0.7279 - val_loss: 1.1920 - val_acc: 0.7314
Epoch 5/20
3896/3896 [==============================] - 2s 629us/step - loss: 1.0517 - acc: 0.7744 - val_loss: 0.9756 - val_acc: 0.7814
Epoch 6/20
3896/3896 [==============================] - 2s 514us/step - loss: 0.8723 - acc: 0.8113 - val_loss: 0.8287 - val_acc: 0.8059
Epoch 7/20
3896/3896 [==============================] - 2s 532us/step - loss: 0.7490 - acc: 0.8368 - val_loss: 0.7258 - val_acc: 0.8274
E

Train on 3900 samples, validate on 975 samples
Epoch 1/20
3900/3900 [==============================] - 3s 884us/step - loss: 2.1924 - acc: 0.2564 - val_loss: 2.0702 - val_acc: 0.4226
Epoch 2/20
3900/3900 [==============================] - 2s 578us/step - loss: 1.9432 - acc: 0.4972 - val_loss: 1.7955 - val_acc: 0.5815
Epoch 3/20
3900/3900 [==============================] - 2s 604us/step - loss: 1.6452 - acc: 0.6292 - val_loss: 1.4827 - val_acc: 0.6821
Epoch 4/20
3900/3900 [==============================] - 2s 579us/step - loss: 1.3414 - acc: 0.7128 - val_loss: 1.2028 - val_acc: 0.7169
Epoch 5/20
1680/3900 [===========>..................] - ETA: 1s - loss: 1.1309 - acc: 0.7583

In [13]:
y_pred=net.model.predict(net.x_test)
print(y_pred)
y_true=net.y_test
print(net.y_test)
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, classification_report
matrix = confusion_matrix(y_true, y_pred.argmax(axis=1))
print(matrix)

[[0.01514964 0.09838519 0.04320014 ... 0.6064487  0.06891719 0.07318833]
 [0.0091253  0.66804224 0.03324823 ... 0.01958369 0.13850038 0.01751763]
 [0.01997857 0.04155824 0.29710037 ... 0.00939071 0.37953192 0.02422441]
 ...
 [0.0858889  0.01374451 0.46914336 ... 0.04438535 0.11808003 0.05197614]
 [0.01016338 0.01254855 0.12352161 ... 0.06587429 0.0487644  0.2672508 ]
 [0.04395359 0.02054369 0.10755294 ... 0.09835456 0.05627401 0.17508022]]
[7 1 8 ... 2 8 3]
[[145   0   6   4   0   2   4   1   1   0]
 [  0 155   5   0   0   0   1   1  20   0]
 [  3   3 108   5   2   2  10   0  23   3]
 [  4   7  29  82   0  16   2   3  17   7]
 [  2   4   6   0  48   3   8   7   7  79]
 [ 15  18   7   7   3  59   6   3  18   6]
 [  6  11   4   0   0   1 140   0   1   3]
 [  4  12   2   2   4   6   1 123   3  14]
 [  0   2  25   4   1  11   1   2  93  11]
 [  1   6   4   1   7   2   3  18   9 110]]


In [15]:
print (classification_report(y_true, y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.81      0.89      0.85       163
           1       0.71      0.85      0.77       182
           2       0.55      0.68      0.61       159
           3       0.78      0.49      0.60       167
           4       0.74      0.29      0.42       164
           5       0.58      0.42      0.48       142
           6       0.80      0.84      0.82       166
           7       0.78      0.72      0.75       171
           8       0.48      0.62      0.54       150
           9       0.47      0.68      0.56       161

   micro avg       0.65      0.65      0.65      1625
   macro avg       0.67      0.65      0.64      1625
weighted avg       0.67      0.65      0.65      1625

